<a href="https://colab.research.google.com/github/Arjun9271/Hands_on_llms/blob/main/chapter_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatgpt for text-classification

### Install the dependencies

In [1]:
! pip install datasets openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Data

In [ ]:
from datasets import load_dataset

data = load_dataset('rotten_tomatoes')
data

# Configuring Openai key

In [2]:
import openai

# create client

client = openai.OpenAI(api_key = " ")

# Chatgeneration Function

In [19]:
import time
import openai

def chatgpt_generation(prompt, document, model="gpt-3.5-turbo", max_retries= 5):

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt.replace("[DOCUMENT]", document)}
    ]

    retries = 0
    while retries < max_retries:
        try:
            # Make the API call
            chat_completion = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=0
            )
            # Return the model's response
            return chat_completion.choices[0].message["content"]

        except openai.RateLimitError:
            retries += 1
            wait_time = 2 ** retries  # Exponential backoff
            print(f"Rate limit exceeded. Retrying in {wait_time} seconds... ({retries}/{max_retries})")
            time.sleep(wait_time)  # Wait before retrying

        except Exception as e:
            print(f"An error occurred: {e}")
            break

    print("Failed to generate a response after retries.")
    return None


# Prompt

In [20]:
# Define a prompt template as a base
prompt = """Predict whether the following document is a positive or negative movie review:

[DOCUMENT]

If it is positive return 1 and if it is negative return 0. Do not give any other answers.
"""

# Predict the target using GPT
document = "unpretentious , charming , quirky , original"
chatgpt_generation(prompt, document)

Rate limit exceeded. Retrying in 2 seconds... (1/5)
Rate limit exceeded. Retrying in 4 seconds... (2/5)
Rate limit exceeded. Retrying in 8 seconds... (3/5)
Rate limit exceeded. Retrying in 16 seconds... (4/5)
Rate limit exceeded. Retrying in 32 seconds... (5/5)
Failed to generate a response after retries.


In [23]:
# Extract predictions
y_pred = [int(pred) for pred in predictions]

# Evaluate performance
evaluate_performance(data["test"]["label"], y_pred)

# Evaluation

In [22]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
    """Create and print the classification report"""
    performance = classification_report(
        y_true, y_pred,
        target_names=["Negative Review", "Positive Review"]
    )
    print(performance)

In [23]:
# Extract predictions
y_pred = [int(pred) for pred in predictions]

# Evaluate performance
evaluate_performance(data["test"]["label"], y_pred)